# LINE-1转座子活性相关lncRNA的高通量筛选与功能初步注释

## 项目概述

本项目基于公共测序数据，快速筛选调控LINE-1转座子的lncRNA，进行表达关联、保守性分析与调控网络预测。

**核心目标：**
- 筛选调控LINE-1转座子的lncRNA
- 分析表达关联和保守性
- 构建调控网络预测
- 输出可复现的分析报告

**适配课题组：** 清华刘念课题组（转座子与RNA调控）

**预计耗时：** 1-2天

## 环境设置与数据准备

In [ ]:
# 导入必要的库
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 设置项目路径
# 兼容从项目根目录或 notebooks/ 目录启动的情况
project_root = Path().absolute()
if not (project_root / "src").exists() and (project_root.parent / "src").exists():
    project_root = project_root.parent
sys.path.append(str(project_root / 'src'))

# 导入项目模块
from data_download import DataDownloader
from differential_expression import DifferentialExpression
from sequence_analysis import SequenceAnalyzer
from network_analysis import NetworkAnalyzer
from visualization import Visualizer

# 设置绘图参数
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

print("✓ 环境设置完成")

In [ ]:
# 加载配置文件
config_path = project_root / 'config' / 'config.yaml'
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print("项目配置：")
print(f"- 参考基因组: {config['data']['reference_genome']}")
print(f"- 差异表达阈值: padj < {config['analysis']['differential_expression']['padj_threshold']}")
print(f"- Log2FC阈值: > {config['analysis']['differential_expression']['log2fc_threshold']}")
print(f"- 保守性阈值: PhyloP > {config['analysis']['conservation']['phylop_threshold']}")

## Day 1: 数据获取与核心分析

### 1.1 数据下载与预处理

In [ ]:
# 初始化数据下载器
downloader = DataDownloader(str(config_path))

print("开始数据下载...")

# 下载参考基因组和注释文件
# 注意：这部分可能需要较长时间，建议提前下载
# downloader.download_reference_genome()
# downloader.download_gencode_annotation()
# downloader.download_line1_sequences()

# 下载SRA数据（示例数据）
# sra_ids = config['data']['sra_accessions']['control'] + config['data']['sra_accessions']['treatment']
# downloader.download_sra_data(sra_ids)

print("✓ 数据下载模块准备完成")

### 1.2 质量控制与序列比对

In [ ]:
# 质量控制示例
fastq_files = [
    "data/raw/fastq/SRR1234567_pass_1.fastq.gz",
    "data/raw/fastq/SRR1234568_pass_1.fastq.gz",
    "data/raw/fastq/SRR1234569_pass_1.fastq.gz"
]

# 执行质量控制
for fastq_file in fastq_files:
    if os.path.exists(fastq_file):
        downloader.quality_control(fastq_file)
    else:
        print(f"文件不存在: {fastq_file}")

print("✓ 质量控制完成")

In [ ]:
# 序列比对示例
bam_files = []
clean_fastq_files = [
    "data/processed/SRR1234567_clean.fastq.gz",
    "data/processed/SRR1234568_clean.fastq.gz",
    "data/processed/SRR1234569_clean.fastq.gz"
]

for i, fastq_file in enumerate(clean_fastq_files):
    if os.path.exists(fastq_file):
        bam_file = f"data/processed/sample_{i+1}.bam"
        downloader.align_reads(fastq_file, bam_file)
        bam_files.append(bam_file)

print(f"✓ 序列比对完成，生成 {len(bam_files)} 个BAM文件")

### 1.3 差异表达分析

In [ ]:
# 初始化差异表达分析器
de_analyzer = DifferentialExpression(str(config_path))

# 计算基因表达量
if len(bam_files) > 0:
    gtf_file = config['data']['gtf_annotation']
    counts_file = config['data']['counts_file']
    
    de_analyzer.count_reads(bam_files, gtf_file, counts_file)
    print(f"✓ 表达量计算完成: {counts_file}")
else:
    print("使用示例数据进行分析...")
    # 创建示例计数数据
    sample_counts = pd.DataFrame({
        'Geneid': [f'ENSG00000{i:06d}' for i in range(1000)],
        'control_1': np.random.negative_binomial(10, 0.5, 1000),
        'control_2': np.random.negative_binomial(10, 0.5, 1000),
        'control_3': np.random.negative_binomial(10, 0.5, 1000),
        'L1OE_1': np.random.negative_binomial(15, 0.4, 1000),
        'L1OE_2': np.random.negative_binomial(15, 0.4, 1000),
        'L1OE_3': np.random.negative_binomial(15, 0.4, 1000)
    })
    
    os.makedirs(os.path.dirname(counts_file), exist_ok=True)
    sample_counts.to_csv(counts_file, index=False)
    print(f"✓ 示例计数数据已创建: {counts_file}")

In [ ]:
# 创建并运行DESeq2分析
script_path = de_analyzer.create_deseq2_script(config['data']['counts_file'])
de_analyzer.run_deseq2_analysis(script_path)

# 读取差异分析结果
diff_results = pd.read_csv("results/tables/significant_genes_deseq2.csv", index_col=0)
print(f"✓ 差异表达分析完成，发现 {len(diff_results)} 个显著差异基因")
print(f"上调基因: {len(diff_results[diff_results['log2FoldChange'] > 0])}")
print(f"下调基因: {len(diff_results[diff_results['log2FoldChange'] < 0])}")

In [ ]:
# 筛选差异表达lncRNA
lncrna_diff = de_analyzer.filter_lncrnas(
    "results/tables/significant_genes_deseq2.csv",
    config['data']['gtf_annotation'],
    "results/tables/lncrna_differential.csv"
)

print(f"✓ 筛选出 {len(lncrna_diff)} 个差异表达lncRNA")
display(lncrna_diff.head())

### 1.4 LINE-1关联分析

In [ ]:
# 分析lncRNA与LINE-1的位置关联
line1_associations = de_analyzer.analyze_line1_association(
    "results/tables/lncrna_differential.csv",
    config['data']['line1_bed'],
    config['data']['gtf_annotation'],
    "results/tables/lncrna_line1_associations.csv"
)

print(f"✓ LINE-1关联分析完成，发现 {len(line1_associations)} 个关联关系")
display(line1_associations.head())

## Day 2: 功能注释与可视化

### 2.1 保守性与序列特征分析

In [ ]:
# 初始化序列分析器
seq_analyzer = SequenceAnalyzer(str(config_path))

# 提取lncRNA序列
seq_analyzer.extract_lncrna_sequences(
    "results/tables/lncrna_differential.csv",
    config['data']['gtf_annotation'],
    config['data']['genome_fasta'],
    "data/processed/lncrnas.fa"
)

print("✓ lncRNA序列提取完成")

In [ ]:
# 保守性分析
conservation_scores = seq_analyzer.calculate_conservation_scores(
    "data/processed/lncrnas.fa",
    "results/tables/conservation_scores.csv"
)

if conservation_scores is not None:
    print("✓ 保守性分析完成")
    display(conservation_scores.describe())
else:
    print("保守性分析跳过（缺少pyBigWig）")

In [ ]:
# 二级结构预测
structure_results = seq_analyzer.predict_secondary_structure(
    "data/processed/lncrnas.fa"
)

print("✓ 二级结构预测完成")
display(structure_results.head())

In [ ]:
# 序列特征分析
sequence_features = seq_analyzer.analyze_sequence_features(
    "data/processed/lncrnas.fa",
    "results/tables/sequence_features.csv"
)

print("✓ 序列特征分析完成")
display(sequence_features.describe())

### 2.2 调控网络构建

In [ ]:
# 初始化网络分析器
net_analyzer = NetworkAnalyzer(str(config_path))

# 构建PPI网络
gene_list = lncrna_diff.index.tolist()[:50]  # 取前50个基因作为示例
ppi_network = net_analyzer.build_ppi_network(
    gene_list,
    "results/networks/ppi_network.edgelist"
)

print(f"✓ PPI网络构建完成")
print(f"节点数: {ppi_network.number_of_nodes()}")
print(f"边数: {ppi_network.number_of_edges()}")

In [ ]:
# 构建lncRNA-mRNA共表达网络
lncrna_mrna_network, corr_df = net_analyzer.build_lncrna_mrna_network(
    "results/tables/lncrna_differential.csv",
    "results/tables/significant_genes_deseq2.csv",
    "data/processed/gene_counts.txt",
    "results/networks/lncrna_mrna_network.edgelist"
)

print(f"✓ lncRNA-mRNA网络构建完成")
print(f"节点数: {lncrna_mrna_network.number_of_nodes()}")
print(f"边数: {lncrna_mrna_network.number_of_edges()}")

In [ ]:
# 整合LINE-1关联信息
integrated_network = net_analyzer.integrate_line1_network(
    lncrna_mrna_network,
    line1_associations,
    "results/networks/integrated_network.edgelist"
)

print(f"✓ LINE-1整合完成")
print(f"整合后节点数: {integrated_network.number_of_nodes()}")
print(f"整合后边数: {integrated_network.number_of_edges()}")

In [ ]:
# 识别关键调控因子
network_metrics, key_regulators = net_analyzer.identify_key_regulators(
    integrated_network,
    "results/networks/network_metrics.csv"
)

print("✓ 关键调控因子识别完成")
print("Top 10 关键节点:")
for i, node in enumerate(key_regulators['top_hubs'][:10]):
    print(f"{i+1}. {node}")

### 2.3 可视化分析

In [ ]:
# 初始化可视化器
visualizer = Visualizer(str(config_path))

# 绘制火山图
visualizer.plot_volcano("results/tables/significant_genes_deseq2.csv")

# 绘制热图
visualizer.plot_heatmap("data/processed/gene_counts.txt")

In [ ]:
# 绘制网络图
visualizer.plot_network("results/networks/integrated_network.edgelist")

# 绘制交互式网络图
visualizer.plot_interactive_network("results/networks/integrated_network.edgelist")

In [ ]:
# 绘制保守性分析图
if os.path.exists("results/tables/conservation_scores.csv"):
    visualizer.plot_conservation_scores("results/tables/conservation_scores.csv")

# 绘制序列特征分析图
visualizer.plot_sequence_features("results/tables/sequence_features.csv")

# 绘制二级结构分析图
if os.path.exists("results/structures/secondary_structures.csv"):
    visualizer.plot_secondary_structure_summary("results/structures/secondary_structures.csv")

In [ ]:
# 创建汇总仪表板
visualizer.create_summary_dashboard()

print("✓ 所有可视化图表生成完成")

## 结果总结与关键发现

In [ ]:
# 汇总分析结果
summary_stats = {
    '差异表达基因总数': len(diff_results),
    '差异表达lncRNA数量': len(lncrna_diff),
    'LINE-1关联lncRNA数量': len(line1_associations['lncRNA_id'].unique()),
    '网络节点总数': integrated_network.number_of_nodes(),
    '网络边总数': integrated_network.number_of_edges(),
    '关键调控因子数量': len(key_regulators['top_hubs'])
}

print("=== 项目分析结果汇总 ===")
for key, value in summary_stats.items():
    print(f"{key}: {value}")

In [ ]:
# 识别高优先级lncRNA（用于后续实验验证）
high_priority_lncrnas = []

# 筛选标准：
# 1. 差异表达显著
# 2. 与LINE-1有位置关联
# 3. 在网络中处于关键位置
# 4. 保守性较高

associated_lncrnas = set(line1_associations['lncRNA_id'].unique())
network_hubs = set(key_regulators['top_hubs'])

for lncrna in lncrna_diff.index:
    score = 0
    
    # 差异表达程度
    if lncrna in lncrna_diff.index:
        log2fc = abs(lncrna_diff.loc[lncrna, 'log2FoldChange'])
        score += min(log2fc / 2, 3)  # 最多3分
    
    # LINE-1关联
    if lncrna in associated_lncrnas:
        score += 2
    
    # 网络重要性
    if lncrna in network_hubs:
        score += 2
    
    # 保守性（如果有的话）
    if conservation_scores is not None and lncrna in conservation_scores['gene_id'].values:
        avg_cons = conservation_scores[conservation_scores['gene_id'] == lncrna]['avg_phylop'].iloc[0]
        if avg_cons > 2.0:
            score += 1
    
    if score >= 4:  # 高优先级阈值
        high_priority_lncrnas.append((lncrna, score))

# 按得分排序
high_priority_lncrnas.sort(key=lambda x: x[1], reverse=True)

print("\n=== 高优先级lncRNA推荐列表 ===")
for i, (lncrna, score) in enumerate(high_priority_lncrnas[:10]):
    print(f"{i+1}. {lncrna} (得分: {score:.1f})")

## 湿实验验证建议

In [ ]:
# 为前2个高优先级lncRNA设计RT-qPCR引物
def design_primers(sequence, primer_length=20, product_size_range=(100, 200)):
    """简化的引物设计函数"""
    seq_len = len(sequence)
    
    # 简单的引物选择（实际应使用更复杂的算法）
    forward_primer = sequence[:primer_length]
    reverse_start = seq_len - primer_length
    reverse_primer = sequence[reverse_start:][::-1]  # 反向互补
    
    return {
        'forward': forward_primer,
        'reverse': reverse_primer,
        'product_size': seq_len
    }

print("=== RT-qPCR验证建议 ===")
for i, (lncrna, score) in enumerate(high_priority_lncrnas[:2]):
    print(f"\n{i+1}. 目标lncRNA: {lncrna}")
    print(f"   优先级得分: {score:.1f}")
    print("   建议实验:")
    print("   - RT-qPCR验证表达差异")
    print("   - RNA FISH定位分析")
    print("   - RIP-seq检测结合蛋白")
    print("   - ChIRP-seq检测基因组结合位点")

## 项目交付物清单

In [ ]:
# 生成项目交付物清单
deliverables = {
    "分析报告": [
        "notebooks/LINE1_lncRNA_analysis.ipynb",
        "results/reports/project_summary.pdf"
    ],
    "数据表格": [
        "results/tables/lncrna_differential.csv",
        "results/tables/lncrna_line1_associations.csv",
        "results/tables/conservation_scores.csv",
        "results/tables/sequence_features.csv",
        "results/tables/network_metrics.csv"
    ],
    "可视化图表": [
        "results/figures/volcano_plot.png",
        "results/figures/heatmap.png",
        "results/figures/network.png",
        "results/figures/interactive_network.html",
        "results/figures/summary_dashboard.png"
    ],
    "网络文件": [
        "results/networks/integrated_network.edgelist",
        "results/networks/ppi_network.edgelist",
        "results/networks/lncrna_mrna_network.edgelist"
    ],
    "结构数据": [
        "results/structures/secondary_structures.csv"
    ]
}

print("=== 项目交付物清单 ===")
for category, files in deliverables.items():
    print(f"\n{category}:")
    for file in files:
        status = "✓" if os.path.exists(file) else "○"
        print(f"  {status} {file}")

## 关键科学假设

基于分析结果，提出以下可验证的科学假设：

1. **lncRNA-LINE-1调控假说**: 高优先级lncRNA通过结合LINE-1启动子区域调控其转录活性
2. **网络中心性假说**: 处于网络中心位置的lncRNA在LINE-1活性调控中起关键作用
3. **保守性功能假说**: 高保守性lncRNA可能具有重要的LINE-1调控功能
4. **结构-功能假说**: 特定二级结构的lncRNA更倾向于与LINE-1元件相互作用
5. **表达关联假说**: lncRNA表达水平与LINE-1转座活性呈负相关关系

## 后续改进方向

1. **数据扩展**: 整合更多公共数据集和单细胞测序数据
2. **方法优化**: 使用更先进的差异分析和网络构建算法
3. **实验验证**: 开展湿实验验证计算预测结果
4. **机制研究**: 深入研究lncRNA调控LINE-1的分子机制
5. **临床应用**: 探索在疾病诊断和治疗中的应用潜力

In [ ]:
print("\n" + "="*50)
print("LINE-1 lncRNA项目分析完成!")
print("="*50)
print(f"分析时间: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"项目路径: {project_root}")
print(f"关键发现: 发现 {len(high_priority_lncrnas)} 个高优先度LINE-1相关lncRNA")
print("\n建议下一步:")
print("1. 对高优先级lncRNA进行湿实验验证")
print("2. 深入研究调控机制")
print("3. 探索临床应用潜力")